In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?','!']
with open("intents.json") as file:
    data = json.load(file)
    

for intent in data['intents']:
    
    for pattern in intent['patterns']:
        
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        for i in w:
            words.append(i)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#lemmatize each word
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#sort class
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

33 documents
8 classes ['about', 'complaint', 'createaccount', 'goodbye', 'greeting', 'help', 'name', 'thanks']
52 unique lemmatized words ["'s", 'a', 'about', 'account', 'an', 'anyone', 'are', 'bye', 'call', 'can', 'complaint', 'could', 'create', 'do', 'for', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'later', 'me', 'name', 'need', 'new', 'open', 'please', 'raise', 'see', 'service', 'should', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'to', 'want', 'what', 'whats', 'who', 'you', 'your']


In [21]:
#create training data
training = []

# create an empty array for output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:

    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\temp\ipykernel_12804\3171580251.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [24]:
# Create a model of 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
7/7 [==============================] - 0s 3ms/step - loss: 2.0603 - accuracy: 0.0909
Epoch 2/200
7/7 [==============================] - 0s 3ms/step - loss: 2.0158 - accuracy: 0.2121
Epoch 3/200
7/7 [==============================] - 0s 3ms/step - loss: 1.9184 - accuracy: 0.3636
Epoch 4/200
7/7 [==============================] - 0s 3ms/step - loss: 1.8677 - accuracy: 0.3030
Epoch 5/200
7/7 [==============================] - 0s 5ms/step - loss: 1.8083 - accuracy: 0.2727
Epoch 6/200
7/7 [==============================] - 0s 4ms/step - loss: 1.6140 - accuracy: 0.4848
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5605 - accuracy: 0.4242
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4633 - accuracy: 0.4545
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 1.3836 - accuracy: 0.5758
Epoch 10/200
7/7 [==============================] - 0s 3ms/step - loss: 1.2565 - accuracy: 0.6667
Epoch 11/200
7/7 [===========

7/7 [==============================] - 0s 3ms/step - loss: 0.0709 - accuracy: 1.0000
Epoch 85/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0427 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0508 - accuracy: 1.0000
Epoch 87/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0829 - accuracy: 1.0000
Epoch 88/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0569 - accuracy: 1.0000
Epoch 89/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0773 - accuracy: 0.9697
Epoch 90/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0240 - accuracy: 1.0000
Epoch 91/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0594 - accuracy: 1.0000
Epoch 92/200
7/7 [==============================] - 0s 2ms/step - loss: 0.0267 - accuracy: 1.0000
Epoch 93/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1191 - accuracy: 0.9394
Epoch 94/200
7/7 [===============

7/7 [==============================] - 0s 5ms/step - loss: 0.0225 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0135 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0342 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0385 - accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 175/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0658 - accuracy: 0.9697
Epoch 176/200
7/7 [==============================] - 0s 3ms/step - loss: 0.0644 - accuracy: 1.0000
Epoch 177/200
7/7 [=====